In [ ]:
import sys
# !{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install -U transformers==4.30.0

In [1]:
import sys
print(sys.executable)

/cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/3.10.2/bin/python


In [37]:
# new_paths = ['', '/cvmfs/soft.computecanada.ca/easybuild/python/site-packages', '/cvmfs/soft.computecanada.ca/custom/python/site-packages', '/cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/3.10.2/lib/python310.zip', '/cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/3.10.2/lib/python3.10', '/cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/3.10.2/lib/python3.10/lib-dynload', '/home/alif/.local/lib/python3.10/site-packages', '/scratch/alif/peft/src', '/scratch/alif/lm-evaluation-harness', '/scratch/alif/minOFT', '/cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/3.10.2/lib/python3.10/site-packages', '/cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Compiler/gcc9/opencv/4.8.0/lib/python3.10/site-packages', '/cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/scipy-stack/2023b/lib/python3.10/site-packages', '/cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/ipykernel/2023b/lib/python3.10/site-packages', '/cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Compiler/gcc9/arrow/13.0.0/lib/python3.10/site-packages', '/cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Compiler/gcc9/arrow/13.0.0/lib/python3.10/site-packages/pyarrow-13.0.0-py3.10-linux-x86_64.egg', '/cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/oldest-supported-numpy/2022a/lib/python3.10/site-packages']
# for path in new_paths:
#     if path not in sys.path:
#         sys.path.insert(0, path)

In [32]:
# export JUPYTER_PATH="${JUPYTER_PATH}:/path/to/add/here/"

In [16]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/scratch/alif'
os.environ['HF_DATASETS_CACHE'] = '/scratch/alif'
os.environ['HF_HOME'] = '/scratch/alif'
os.environ['WANDB_MODE'] = 'offline'


In [17]:
import os
import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [18]:
import os
from datasets import load_dataset, Dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import pandas as pd

In [19]:
model_name = "meta-llama/Llama-2-7b-hf"
dataset_name = "timdettmers/openassistant-guanaco"

lora_r = 64
lora_alpha = 16
lora_dropout = 0.1


use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False


output_dir = "./results"
num_train_epochs = 1
fp16 = True
bf16 = False
per_device_train_batch_size = 1
per_device_eval_batch_size = 1
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001


optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True

save_steps = 0
logging_steps = 25


max_seq_length = None
packing = False

In [20]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)
        fp16 = False
        bf16 = True

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    # device_map=device_map,
    cache_dir="/scratch/alif"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, cache_dir="/scratch/alif")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
dataset_path = "/scratch/alif/timdettmers___json/timdettmers--openassistant-guanaco-c93588435bc90172/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e/"
train_path = os.path.join(dataset_path, 'json-train.arrow')
test_path = os.path.join(dataset_path, 'json-test.arrow')

In [22]:
train_dataset = Dataset.from_file(train_path)
test_dataset = Dataset.from_file(test_path)

In [23]:
train_dataset[0]

{'text': '### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining po

In [24]:
# Assuming 'train_dataset' is a list of dictionaries with the key 'text'
texts = [entry['text'] for entry in train_dataset]

# Step 1 & 2: Data Preparation and Feature Extraction
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(texts)

# Step 3: Clustering
kmeans = KMeans(n_clusters=2, random_state=0)
kmeans.fit(X)

# Assigning cluster labels to each text
cluster_labels = kmeans.labels_

# Creating a DataFrame for easier visualization
clustered_data = pd.DataFrame({'text': texts, 'cluster': cluster_labels})

print(clustered_data)

/home/alif/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


                                                   text  cluster
0     ### Human: Can you write a short introduction ...        0
1     ### Human: ¿CUales son las etapas del desarrol...        1
2     ### Human: Can you explain contrastive learnin...        0
3     ### Human: I want to start doing astrophotogra...        0
4     ### Human: Método del Perceptrón biclásico: de...        1
...                                                 ...      ...
9841  ### Human: What are the risks when you do not ...        0
9842  ### Human: 为什么不能关灯玩手机？### Assistant: 出於各種原因，不建...        0
9843  ### Human: ¿De qué está hecho el plástico?### ...        1
9844  ### Human: Nortzuk ziren Harry Potteren guraso...        0
9845  ### Human: Nork jarri zion ahotsa Dragoi Bolak...        0

[9846 rows x 2 columns]


In [25]:
# Assuming 'clustered_data' is your DataFrame with texts and cluster labels

# Split the DataFrame into two based on the cluster labels
cluster_0_df = clustered_data[clustered_data['cluster'] == 0]
cluster_1_df = clustered_data[clustered_data['cluster'] == 1]

# Convert these DataFrames into Hugging Face Dataset objects
cluster_0_dataset = Dataset.from_pandas(cluster_0_df)
cluster_1_dataset = Dataset.from_pandas(cluster_1_df)

In [26]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to=None
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=cluster_0_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/scratch/alif/peft/src/peft/utils/other.py:136: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/home/alif/.local/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/6450 [00:00<?, ? examples/s]

In [ ]:
trainer.train()
new_model = "llama-2-7b-guanaco-cluster0"
trainer.model.save_pretrained(new_model)

Traceback (most recent call last):
  File "/cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/3.10.2/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/3.10.2/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/alif/.local/lib/python3.10/site-packages/wandb/__main__.py", line 3, in <module>
    cli.cli(prog_name="python -m wandb")
  File "/home/alif/.local/lib/python3.10/site-packages/click/core.py", line 1130, in __call__
    return self.main(*args, **kwargs)
  File "/home/alif/.local/lib/python3.10/site-packages/click/core.py", line 1055, in main
    rv = self.invoke(ctx)
  File "/home/alif/.local/lib/python3.10/site-packages/click/core.py", line 1657, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/home/alif/.local/lib/python3.10/site-packages/click/core.py", line 1

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/alif/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
25,1.388500
50,1.660700
75,1.292300
100,1.545400
125,1.094700
150,1.393600
175,1.172500
200,1.442800
225,1.175300
250,1.544800
